### Configuration

In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime

import h5py
import hdf5storage
from tqdm import tqdm

#from utils__helpers_micro import mat_to_npy
#from utils__helpers_micro import npy_to_si
import utils__config as config

In [2]:
os.chdir(config.working_directory)
os.getcwd()

'g:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep'

### Parameters

In [3]:
dir_base = 'M:/Sleep/Cache/Subject02/Apr26'
dir_from = 'S02_Apr26_micro__1___ns5_to_mat'
dir_to = 'S02_Apr26_micro__2___mat_to_mat'

tmin = datetime.datetime(2022, 4, 27, 1, 59, 20, 0, tzinfo = datetime.timezone.utc)
tmax = datetime.datetime(2022, 4, 27, 7, 37, 20, 0, tzinfo = datetime.timezone.utc)

dictionary_path = 'Data/Subject02/S02_dictionary.xlsx'

In [4]:
# Load channel dictionary
ch_dictionary = pd.read_excel(dictionary_path)

# Get list of files and sort by channel number
file_list = os.listdir(os.path.join(dir_base, dir_from))
file_list.sort()

# Select only the subset of files you want
#file_list = file_list[0:1] 
#print(len(file_list))

### Define Function

In [5]:
def mat_to_npy(file_path, start=None, stop=None):
    # Import MAT file
    data = hdf5storage.loadmat(file_path) # , variable_names = ['foo', 'bar']

    # Extract sampling frequency
    sampling_rate = data['meta_data']['sampling_rate'].astype(np.int64)[0][0][0]

    # NSx raw timestamps are formatted YYYY-MM-?-DD-HH-MM-SS-MSMSMS
    # (so you do not use the third element in the timestamp; 
    #  also, MNE only accepts timestamps in the UTC timezone)
    raw_time = data['meta_data']['time_stamp'][0][0][0][0].astype(np.int64)
    time_start = datetime.datetime(raw_time[0], raw_time[1], raw_time[3], 
                                   raw_time[4], raw_time[5], raw_time[6], 
                                   raw_time[7], tzinfo = datetime.timezone.utc)

    # Convert time series data from uV to Volts
    data = data['time_series'][0] * 1e-6

    # Since this is only one channel, it loads as only one dimension. We need to add a 
    # dummy second dimension (channels) for se.NumpyRecording() to work.
    data = data[:, np.newaxis]

    # If start and stop are provided, crop the data
    if start is not None and stop is not None:
        # Convert tmin/tmax to sample numbers
        tmin = (start - time_start).total_seconds() * sampling_rate
        tmax = (stop - time_start).total_seconds() * sampling_rate

        # Crop the data to the relevant time
        data = data[tmin.astype(np.int64) : tmax.astype(np.int64) + 1, :]

    return data, sampling_rate

### Munging

In [6]:
for file in tqdm(file_list):
    # Import .MAT files into Python and format into Numpy objects
    # (this will also crop the data to the relevant times)
    file_path = os.path.join(dir_base, dir_from, file)
    data, sampling_rate = mat_to_npy(file_path, tmin, tmax)

    # Extract the channel number from the file name
    file_name = os.path.splitext(file)[0]
    regex = re.compile('_Channel([0-9]*)')
    regex = regex.findall(file_name)
    regex = int(regex[0])

    # Add region and laterality to the file name
    name_columns = ['laterality', 'region']
    file_suffix = ch_dictionary.loc[ch_dictionary['number'] == regex, name_columns]
    file_suffix = '_' + file_suffix.iloc[0]['laterality'] + '_' + file_suffix.iloc[0]['region']

    file_save_path = os.path.join(dir_base, dir_to, file_name + file_suffix + '.mat')
    print("Saving to: ", file_save_path)

    # Please note that the field can ONLY BE NAMED "data"
    # because Waveclus will only recognize MAT files where
    # the only field is named "data".
    hdf5storage.savemat(file_name = file_save_path, 
                        mdict = {'data' : data}, 
                        format = '7.3')

  0%|          | 0/65 [00:00<?, ?it/s]

Saving to:  M:/Sleep/Cache/Subject02/Apr26\S02_Apr26_micro__2___mat_to_mat\S02_Channel193_right_CLA.mat


  2%|▏         | 1/65 [05:39<6:02:00, 339.38s/it]

Saving to:  M:/Sleep/Cache/Subject02/Apr26\S02_Apr26_micro__2___mat_to_mat\S02_Channel194_right_CLA.mat


  3%|▎         | 2/65 [11:21<5:58:06, 341.05s/it]

...

Saving to:  M:/Sleep/Cache/Subject02/Apr26\S02_Apr26_micro__2___mat_to_mat\S02_Channel255_left_AMY.mat


 97%|█████████▋| 63/65 [5:53:01<10:53, 326.69s/it]

Saving to:  M:/Sleep/Cache/Subject02/Apr26\S02_Apr26_micro__2___mat_to_mat\S02_Channel256_left_AMY.mat


 98%|█████████▊| 64/65 [5:58:48<05:36, 336.39s/it]


TypeError: can only concatenate str (not "float") to str